In [1]:
# Load kedro environment (not needed in .py)
from pathlib import Path
from kedro.framework.context import load_context

# Load a context to be able to work in the notebook
#current_dir = Path.cwd()
current_dir = Path("/u01/share/cesar/aa_engine_uy/notebooks/")
proj_path = current_dir.parent
context = load_context(proj_path)
catalog = context.catalog
credentials = context.config_loader.get("credentials*","credentials*/**")
parameters = context.config_loader.get("parameters*","parameters*/**")

from aa_engine_pkg.assets.utils import *
from aa_engine_pkg.assets.core.data.kedro.catalog_expansion.partitioned_sql import SQLPartitionedDataSet

In [2]:
date='20180605'

In [3]:
def create_cliente_activo(cliente_activo: SQLPartitionedDataSet,
                          date: str) -> pd.DataFrame:
    """Creates master table with features related to EoP state of customers for one period of data

    Parameters
    ----------
    cliente_activo:
        dataset defined in ´catalog.yml´ - list of active customers at EoP for the given period
    date:
        period to process
    
    Returns
    -------
        Mastertable with information of clientes at EoP
    """

    # Initialize logger
    log = initialize_logger()

    # Load active clientes for period
    log.info(f"Creating cliente_activo...")
    period_to_load = get_previous_month(date)
    df_clientes_activos = cliente_activo.filter_by(date=period_to_load)

    # Return
    return df_clientes_activos

In [4]:
cliente_activo=catalog.load("cliente_activo")

2020-12-30 19:21:53,806 - kedro.io.data_catalog - INFO - Loading data from `cliente_activo` (SQLPartitionedDataSet)...


In [5]:
cliente_activo_df= create_cliente_activo(cliente_activo,date)

2020-12-30 19:21:54,509 - aa_engine_pkg.assets.utils.utilities - INFO - Creating cliente_activo...
select distinct CUSTOMER_ID from stg_uy_customer_status where UPPER(STATUS) LIKE '%ACTIVO%' and DATE_EXP = 201805


/u01/miniconda3/envs/uypo37/lib/python3.7/site-packages/sqlalchemy/dialects/oracle/base.py:1381: SAWarning: Oracle version (19, 5, 0, 0, 0) is known to have a maximum identifier length of 128, rather than the historical default of 30. SQLAlchemy 1.4 will use 128 for this database; please set max_identifier_length=128 in create_engine() in order to test the application with this new length, or set to 30 in order to assure that 30 continues to be used.  In particular, pay close attention to the behavior of database migrations as dynamically generated names may change. See the section 'Max Identifier Lengths' in the SQLAlchemy Oracle dialect documentation for background.
  % ((self.server_version_info,))


In [6]:
mantenimiento=catalog.load("mantenimiento")

2020-12-30 19:22:09,314 - kedro.io.data_catalog - INFO - Loading data from `mantenimiento` (SQLPartitionedDataSet)...


In [7]:
past_periods = [14, 28, 84, 168]

string_vars = ["ESTADOWO", "SUBSTYPENAME"]

expand_vars = ["FLAG_CORPORATIVOS", "FLAG_COMPLETED", "TIME_TO_COMPLETE", "FECHA_CREACION", "CUSTOMER_ID", "ID"]

categories_dict = {"SERVICIO_ID": ["S", "IA", "SC"],
                   "DEALER_WO_ID": ["PERSONAL", "MIXER", "UNIVISION", "GLOBAL", "TIME"]}

vars_to_dummy = ["SERVICIO_ID", "DEALER_WO_ID", "ESTADOWO_ID", "SUBSTYPENAME_ID"]

In [8]:
# Initialize logger
log = initialize_logger()

In [10]:
# Read parameters
lookback_days = parameters["masters"]["global"]["look_back_days"]
start_date = pd.to_datetime(date) - timedelta(days=lookback_days)

# Calculate period to load for active clients
log.info("Loading EoP clients...")
period_to_load = get_previous_month(date)
df_clientes = cliente_activo_df[["CUSTOMER_ID"]]

# Get mantenimiento table
log.info("Loading Mantenimiento table...")
df_m = mantenimiento.filter_by(date=[start_date.strftime("%Y%m%d"),
                                     date])

# Merging
log.info("Merging Mantenimiento with EoP table...")
df_m = df_clientes.merge(df_m,
                         on="CUSTOMER_ID",
                         how="inner",
                         validate="1:m")

# Format string variables
df_m["SERVICIO_ID"] = df_m["SERVICIO"].str.extract(r'([a-zA-Z]+)\d+')
log.info("Formatting varibles...")
for var in string_vars:
    new_name = var + "_ID"
    df_m[new_name] = df_m[var].str.split("-").str[0].str.strip()

2020-12-30 19:24:42,974 - aa_engine_pkg.assets.utils.utilities - INFO - Loading EoP clients...
2020-12-30 19:24:42,978 - aa_engine_pkg.assets.utils.utilities - INFO - Loading Mantenimiento table...
select CUSTOMER_ID, DATE_EXP, ID, FECHA_CREACION, ESTADOWO, COMPLETED_DATE_TIME, SHUSERNR, SUBSTYPENAME, PROVINCIA, KEYWORD, SERVICIO, DEALER_WO from stg_uy_mantenimiento where FECHA_CREACION >= to_date('20171219', 'yyyymmdd') and FECHA_CREACION < to_date('20180605', 'yyyymmdd')
2020-12-30 19:24:45,965 - aa_engine_pkg.assets.utils.utilities - INFO - Merging Mantenimiento with EoP table...
2020-12-30 19:24:46,374 - aa_engine_pkg.assets.utils.utilities - INFO - Formatting varibles...


In [11]:
df_m.head()

,CUSTOMER_ID,DATE_EXP,ID,FECHA_CREACION,ESTADOWO,COMPLETED_DATE_TIME,SHUSERNR,SUBSTYPENAME,PROVINCIA,KEYWORD,SERVICIO,DEALER_WO,SERVICIO_ID,ESTADOWO_ID,SUBSTYPENAME_ID
0,146099,201804,1289946,2018-04-14 09:54:03,C- Cancelada,NaT,8026,N - Normal Casa,MONTEVIDEO,None,LO03-Visita Pre instalacion,PLAZA CORREO,LO,C,N
1,146099,201804,1289947,2018-04-14 09:54:04,C- Cancelada,NaT,8026,N - Normal Casa,MONTEVIDEO,None,LO03-Visita Pre instalacion,PLAZA CORREO,LO,C,N
2,146099,201804,1289948,2018-04-14 09:55:03,C- Cancelada,NaT,8026,N - Normal Casa,MONTEVIDEO,None,LO03-Visita Pre instalacion,PLAZA CORREO,LO,C,N
3,149934,201801,1252303,2018-01-31 22:25:34,F- Finalizada,2018-02-02 09:10:48,9078,N - Normal Casa,CANELONES,None,S06-Problemas del IRD,JOVE SRL,S,F,N
4,149934,201801,1252303,2018-01-31 22:25:34,F- Finalizada,2018-02-02 09:10:48,9078,N - Normal Casa,CANELONES,None,SD02-DVR: Reemplazo por mal funcionamiento,JOVE SRL,SD,F,N


In [12]:
# Format string variables
new_name = "DEALER_WO" + "_ID"
df_m[new_name] = df_m["DEALER_WO"].str.split(" ").str[1]

In [13]:
df_m.head()

,CUSTOMER_ID,DATE_EXP,ID,FECHA_CREACION,ESTADOWO,COMPLETED_DATE_TIME,SHUSERNR,SUBSTYPENAME,PROVINCIA,KEYWORD,SERVICIO,DEALER_WO,SERVICIO_ID,ESTADOWO_ID,SUBSTYPENAME_ID,DEALER_WO_ID
0,146099,201804,1289946,2018-04-14 09:54:03,C- Cancelada,NaT,8026,N - Normal Casa,MONTEVIDEO,None,LO03-Visita Pre instalacion,PLAZA CORREO,LO,C,N,CORREO
1,146099,201804,1289947,2018-04-14 09:54:04,C- Cancelada,NaT,8026,N - Normal Casa,MONTEVIDEO,None,LO03-Visita Pre instalacion,PLAZA CORREO,LO,C,N,CORREO
2,146099,201804,1289948,2018-04-14 09:55:03,C- Cancelada,NaT,8026,N - Normal Casa,MONTEVIDEO,None,LO03-Visita Pre instalacion,PLAZA CORREO,LO,C,N,CORREO
3,149934,201801,1252303,2018-01-31 22:25:34,F- Finalizada,2018-02-02 09:10:48,9078,N - Normal Casa,CANELONES,None,S06-Problemas del IRD,JOVE SRL,S,F,N,SRL
4,149934,201801,1252303,2018-01-31 22:25:34,F- Finalizada,2018-02-02 09:10:48,9078,N - Normal Casa,CANELONES,None,SD02-DVR: Reemplazo por mal funcionamiento,JOVE SRL,SD,F,N,SRL


In [14]:
# Format date variable
df_m["FECHA_CREACION"] = df_m["FECHA_CREACION"].dt.strftime("%Y%m%d")

# Flags and time to completion of WO
condition = (df_m["KEYWORD"] == "EC-CORPORATIVOS")
df_m['FLAG_CORPORATIVOS'] = np.where(condition, 1, 0)
df_m['FLAG_COMPLETED'] = np.where(df_m['COMPLETED_DATE_TIME'], 0, 1)
df_m["TIME_TO_COMPLETE"] = (df_m['COMPLETED_DATE_TIME'] - pd.to_datetime(
    df_m["FECHA_CREACION"])) / np.timedelta64(1, "D")

In [15]:
df_m.head()

,CUSTOMER_ID,DATE_EXP,ID,FECHA_CREACION,ESTADOWO,COMPLETED_DATE_TIME,SHUSERNR,SUBSTYPENAME,PROVINCIA,KEYWORD,SERVICIO,DEALER_WO,SERVICIO_ID,ESTADOWO_ID,SUBSTYPENAME_ID,DEALER_WO_ID,FLAG_CORPORATIVOS,FLAG_COMPLETED,TIME_TO_COMPLETE
0,146099,201804,1289946,20180414,C- Cancelada,NaT,8026,N - Normal Casa,MONTEVIDEO,None,LO03-Visita Pre instalacion,PLAZA CORREO,LO,C,N,CORREO,0,0,NaN
1,146099,201804,1289947,20180414,C- Cancelada,NaT,8026,N - Normal Casa,MONTEVIDEO,None,LO03-Visita Pre instalacion,PLAZA CORREO,LO,C,N,CORREO,0,0,NaN
2,146099,201804,1289948,20180414,C- Cancelada,NaT,8026,N - Normal Casa,MONTEVIDEO,None,LO03-Visita Pre instalacion,PLAZA CORREO,LO,C,N,CORREO,0,0,NaN
3,149934,201801,1252303,20180131,F- Finalizada,2018-02-02 09:10:48,9078,N - Normal Casa,CANELONES,None,S06-Problemas del IRD,JOVE SRL,S,F,N,SRL,0,0,2.3825
4,149934,201801,1252303,20180131,F- Finalizada,2018-02-02 09:10:48,9078,N - Normal Casa,CANELONES,None,SD02-DVR: Reemplazo por mal funcionamiento,JOVE SRL,SD,F,N,SRL,0,0,2.3825


In [16]:
# Flatten by customer_id and date

df_comp = df_m[expand_vars]
df_flatten = df_comp.groupby(["CUSTOMER_ID", "FECHA_CREACION"]).agg({"FLAG_CORPORATIVOS": np.nansum,
                                                                     "TIME_TO_COMPLETE": np.nanmax,
                                                                     "ID": "nunique"}).reset_index()
df_flatten.rename(columns={"FLAG_CORPORATIVOS": "N_CORPORATIVOS",
                           "TIME_TO_COMPLETE": "DAYS_TO_COMPLETE",
                           "ID": "N_WO"}
                  , inplace=True)

log.info("Creating past variables...")
df_past = add_relative_calculate_past(df=df_flatten,
                                      id_cols=["CUSTOMER_ID"],
                                      periods=past_periods,
                                      agg={"N_CORPORATIVOS": np.nansum,
                                           "N_WO": np.nansum,
                                           "DAYS_TO_COMPLETE": [np.nanmin, np.nanmax]},
                                      date_col="FECHA_CREACION",
                                      start_date=start_date,
                                      end_date=date,
                                      period_freq="D")
# Impute categories
impute_categories(df_m,
                  "SERVICIO_ID",
                  categories_dict["SERVICIO_ID"]
                  )

impute_categories(df_m,
                  "DEALER_WO_ID",
                  categories_dict["DEALER_WO_ID"]
                  )

log.info("Creating dummy variables...")
df = []
for var in vars_to_dummy:
    log.info(f'---- {var}')
    df.append(create_dummy_variables(df=df_m,
                                     vars_to_groupby=["CUSTOMER_ID", "FECHA_CREACION"],
                                     var_to_dummy=var,
                                     include_total=False,
                                     include_subtotal=False))

mantenimiento_dummies = reduce(
    lambda left, right: pd.merge(left, right, on=["CUSTOMER_ID", "FECHA_CREACION"], how="outer"), df)

# Create expanded variables
log.info("Creating past variables...")
df_num = add_relative_calculate_past(df=mantenimiento_dummies,
                                     id_cols=["CUSTOMER_ID"],
                                     periods=past_periods,
                                     agg=[np.nansum],
                                     date_col="FECHA_CREACION",
                                     start_date=start_date,
                                     end_date=date,
                                     period_freq="D")

# Merging
log.info("Merging the two past variables tables...")
df_master_mantenimiento = df_past.merge(df_num,
                                        on="CUSTOMER_ID",
                                        how="inner",
                                        validate="1:1")

# Add date variables
df_master_mantenimiento["DATE_EXP"] = period_to_load
df_master_mantenimiento["DATE_CALC"] = date
id_cols = ["CUSTOMER_ID", "DATE_EXP", "DATE_CALC"]

# Change variable names 
table_preffix = parameters["masters"]["mantenimiento"]["table_preffix"]
rename_table(df_master_mantenimiento,
             preffix=table_preffix,
             ids_to_exclude=id_cols)

2020-12-30 19:27:58,431 - aa_engine_pkg.assets.utils.utilities - INFO - Creating past variables...
2020-12-30 19:27:58,433 - aa_engine_pkg.assets.utils.utilities - INFO - Adding relative date between 2017-12-19 00:00:00 and 20180605
2020-12-30 19:27:58,454 - aa_engine_pkg.assets.utils.utilities - INFO - Expanding table for period: 14
2020-12-30 19:27:58,470 - aa_engine_pkg.assets.utils.utilities - INFO - Expanding table for period: 28
2020-12-30 19:27:58,490 - aa_engine_pkg.assets.utils.utilities - INFO - Expanding table for period: 84
2020-12-30 19:27:58,519 - aa_engine_pkg.assets.utils.utilities - INFO - Expanding table for period: 168
2020-12-30 19:27:58,733 - aa_engine_pkg.assets.utils.utilities - INFO - Creating dummy variables...
2020-12-30 19:27:58,735 - aa_engine_pkg.assets.utils.utilities - INFO - ---- SERVICIO_ID
2020-12-30 19:27:58,777 - aa_engine_pkg.assets.utils.utilities - INFO - ---- DEALER_WO_ID
2020-12-30 19:27:58,814 - aa_engine_pkg.assets.utils.utilities - INFO - ---

In [18]:
df_master_mantenimiento.head()

,CUSTOMER_ID,MAN_N_CORPORATIVOS_nansum_14,MAN_N_WO_nansum_14,MAN_DAYS_TO_COMPLETE_nanmin_14,MAN_DAYS_TO_COMPLETE_nanmax_14,MAN_N_CORPORATIVOS_nansum_28,MAN_N_WO_nansum_28,MAN_DAYS_TO_COMPLETE_nanmin_28,MAN_DAYS_TO_COMPLETE_nanmax_28,MAN_N_CORPORATIVOS_nansum_84,...,MAN_SUBSTYPENAME_ID_N_nansum_168,MAN_SERVICIO_ID_OTHER_nansum_168,MAN_SERVICIO_ID_IA_nansum_168,MAN_ESTADOWO_ID_F_nansum_168,MAN_ESTADOWO_ID_T_nansum_168,MAN_SUBSTYPENAME_ID_D_nansum_168,MAN_DEALER_WO_ID_OTHER_nansum_168,MAN_ESTADOWO_ID_C_nansum_168,DATE_EXP,DATE_CALC
0,8176,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,...,1,1,0,1,0,0,1,0,201805,20180605
1,8855,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,...,1,1,0,1,0,0,1,0,201805,20180605
2,8864,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,...,0,2,0,2,0,2,2,1,201805,20180605
3,8882,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,...,0,1,0,0,0,1,1,1,201805,20180605
4,9331,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,...,1,1,0,1,0,0,1,0,201805,20180605


In [ ]:
def create_master_mantenimiento(mantenimiento: SQLPartitionedDataSet,
                                cliente_activo: pd.DataFrame,
                                parameters: Dict,
                                date: str) -> pd.DataFrame:
    """Creates master table with features related to interactions with call center for one period of data
    Parameters
    ----------
    cliente_activo: 
        dataset defined in ``catalog_raw.yml`` - list of active customers at EoP for the given period
    mantenimiento:
        dataset defined in ``catalog_raw.yml`` with raw data information related to work orders
    date: 
        period to process
    parameters: 
        set of project parameters defined in ``parameters.yml``
    Returns
    -------
    pd.DataFrame
        Master table with call center interactions features for one period
    """

    # Initialize logger
    log = initialize_logger()

    write_to_parquet = parameters["write_to_parquet"]
    table_name = "mantenimiento"
    overwrite = parameters["masters"][table_name]["overwrite"]

    # Check if table was already created
    files = get_mastertable_paths_by_period(parameters=parameters, period=date)
    match = [str(file) for file in files if table_name in file]

    if len(match) > 0 and overwrite is False:
        # If table is found, read parquet:
        log.info(f"Reading {match[0]} table")
        df_master_mantenimiento = pd.read_parquet(match[0], engine="pyarrow")

    else:
        log.info("Creating master table mantenimiento")

        # Read parameters
        lookback_days = parameters["masters"]["global"]["look_back_days"]
        start_date = pd.to_datetime(date) - timedelta(days=lookback_days)

        # Calculate period to load for active clients
        log.info("Loading EoP clients...")
        period_to_load = get_previous_month(date)
        df_clientes = cliente_activo[["CUSTOMER_ID"]]

        # Get mantenimiento table
        log.info("Loading Mantenimiento table...")
        df_m = mantenimiento.filter_by(date=[start_date.strftime("%Y%m%d"),
                                             date])

        # Merging
        log.info("Merging Mantenimiento with EoP table...")
        df_m = df_clientes.merge(df_m,
                                 on="CUSTOMER_ID",
                                 how="inner",
                                 validate="1:m")

        # Format string variables
        df_m["SERVICIO_ID"] = df_m["SERVICIO"].str.extract(r'([a-zA-Z]+)\d+')
        log.info("Formatting varibles...")
        for var in string_vars:
            new_name = var + "_ID"
            df_m[new_name] = df_m[var].str.split("-").str[0].str.strip()

        # Format string variables
        new_name = "DEALER_WO" + "_ID"
        df_m[new_name] = df_m["DEALER_WO"].str.split(" ").str[1]

        # Format date variable
        df_m["FECHA_CREACION"] = df_m["FECHA_CREACION"].dt.strftime("%Y%m%d")

        # Flags and time to completion of WO
        condition = (df_m["KEYWORD"] == "EC-CORPORATIVOS")
        df_m['FLAG_CORPORATIVOS'] = np.where(condition, 1, 0)
        df_m['FLAG_COMPLETED'] = np.where(df_m['COMPLETED_DATE_TIME'], 0, 1)
        df_m["TIME_TO_COMPLETE"] = (df_m['COMPLETED_DATE_TIME'] - pd.to_datetime(
            df_m["FECHA_CREACION"])) / np.timedelta64(1, "D")

        # Flatten by customer_id and date

        df_comp = df_m[expand_vars]
        df_flatten = df_comp.groupby(["CUSTOMER_ID", "FECHA_CREACION"]).agg({"FLAG_CORPORATIVOS": np.nansum,
                                                                             "TIME_TO_COMPLETE": np.nanmax,
                                                                             "ID": "nunique"}).reset_index()
        df_flatten.rename(columns={"FLAG_CORPORATIVOS": "N_CORPORATIVOS",
                                   "TIME_TO_COMPLETE": "DAYS_TO_COMPLETE",
                                   "ID": "N_WO"}
                          , inplace=True)

        log.info("Creating past variables...")
        df_past = add_relative_calculate_past(df=df_flatten,
                                              id_cols=["CUSTOMER_ID"],
                                              periods=past_periods,
                                              agg={"N_CORPORATIVOS": np.nansum,
                                                   "N_WO": np.nansum,
                                                   "DAYS_TO_COMPLETE": [np.nanmin, np.nanmax]},
                                              date_col="FECHA_CREACION",
                                              start_date=start_date,
                                              end_date=date,
                                              period_freq="D")
        # Impute categories
        impute_categories(df_m,
                          "SERVICIO_ID",
                          categories_dict["SERVICIO_ID"]
                          )

        impute_categories(df_m,
                          "DEALER_WO_ID",
                          categories_dict["DEALER_WO_ID"]
                          )

        log.info("Creating dummy variables...")
        df = []
        for var in vars_to_dummy:
            log.info(f'---- {var}')
            df.append(create_dummy_variables(df=df_m,
                                             vars_to_groupby=["CUSTOMER_ID", "FECHA_CREACION"],
                                             var_to_dummy=var,
                                             include_total=False,
                                             include_subtotal=False))

        mantenimiento_dummies = reduce(
            lambda left, right: pd.merge(left, right, on=["CUSTOMER_ID", "FECHA_CREACION"], how="outer"), df)

        # Create expanded variables
        log.info("Creating past variables...")
        df_num = add_relative_calculate_past(df=mantenimiento_dummies,
                                             id_cols=["CUSTOMER_ID"],
                                             periods=past_periods,
                                             agg=[np.nansum],
                                             date_col="FECHA_CREACION",
                                             start_date=start_date,
                                             end_date=date,
                                             period_freq="D")

        # Merging
        log.info("Merging the two past variables tables...")
        df_master_mantenimiento = df_past.merge(df_num,
                                                on="CUSTOMER_ID",
                                                how="inner",
                                                validate="1:1")

        # Add date variables
        df_master_mantenimiento["DATE_EXP"] = period_to_load
        df_master_mantenimiento["DATE_CALC"] = date
        id_cols = ["CUSTOMER_ID", "DATE_EXP", "DATE_CALC"]

        # Change variable names 
        table_preffix = parameters["masters"]["mantenimiento"]["table_preffix"]
        rename_table(df_master_mantenimiento,
                     preffix=table_preffix,
                     ids_to_exclude=id_cols)
        log.info(f"Exporting {df_master_mantenimiento.shape[0]} rows and {df_master_mantenimiento.shape[1]} columns")

        if write_to_parquet:
            log.info(f"\n------ Writing {table_name} for period {date} to parquet ------")
            file = f"{parameters['paths']['master_path']}master_{table_name}/master_{table_name}_{date}.parquet"
            df_master_mantenimiento.to_parquet(file, engine="pyarrow")

        log.info(f"Exporting {df_master_mantenimiento.shape[0]} rows and {df_master_mantenimiento.shape[1]} columns")

    return df_master_mantenimiento